In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin

from tensorflow.python.eager import context
from tensorflow.python.framework import common_shapes
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import ops
from tensorflow.python.framework import sparse_tensor
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import gen_control_flow_ops
from tensorflow.python.ops import gen_data_flow_ops
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import gen_sparse_ops
from tensorflow.python.ops import gen_spectral_ops
from tensorflow.python.ops import gen_state_ops
from tensorflow.python.ops import state_ops
# go/tf-wildcard-import
# pylint: disable=wildcard-import
from tensorflow.python.ops.gen_math_ops import *
# pylint: enable=wildcard-import
from tensorflow.python.util import compat
from tensorflow.python.util import deprecation

# Aliases for some automatically-generated names.
linspace = gen_math_ops.lin_space

arg_max = deprecation.deprecated(None, "Use `argmax` instead")(arg_max)  # pylint: disable=used-before-assignment
arg_min = deprecation.deprecated(None, "Use `argmin` instead")(arg_min)  # pylint: disable=used-before-assignment


In [39]:
import tensorflow as tf
shape=[3,3,3]
a  = tf.constant([[0,1.0,2.0],[0,0,0],[0,0,0]])
b  = tf.placeholder("float", shape=shape)
name = None
axes = 1

In [33]:
tf.tensordot(a,b,axes=1)

<tf.Tensor 'Tensordot_6:0' shape=<unknown> dtype=float32>

In [19]:
def _tensordot_axes(a, axes):
    """Generates two sets of contraction axes for the two tensor arguments."""
    a_shape = a.get_shape()
    if isinstance(axes, compat.integral_types):
        if axes < 1:
            raise ValueError("'axes' must be at least 1.")
        if a_shape.ndims is not None:
            return range(a_shape.ndims - axes, a_shape.ndims), range(axes)
        else:
            rank = array_ops.rank(a)
            return (range(rank - axes, rank, dtype=dtypes.int32),
                            range(axes, dtype=dtypes.int32))
    elif isinstance(axes, (list, tuple)):
        if len(axes) != 2:
            raise ValueError("'axes' must be an integer or have length 2.")
        a_axes = axes[0]
        b_axes = axes[1]
        if isinstance(a_axes, compat.integral_types) and \
                isinstance(b_axes, compat.integral_types):
            a_axes = [a_axes]
            b_axes = [b_axes]
        if len(a_axes) != len(b_axes):
            raise ValueError(
                    "Different number of contraction axes 'a' and 'b', %s != %s." %
                    (len(a_axes), len(b_axes)))
        return a_axes, b_axes
    else:
        axes = ops.convert_to_tensor(axes, name="axes", dtype=dtypes.int32)
        return axes[0], axes[1]

In [34]:
def _tensordot_reshape(a, axes, flipped=False):
    """Helper method to perform transpose and reshape for contraction op.

    This method is helpful in reducing `math_ops.tensordot` to `math_ops.matmul`
    using `array_ops.transpose` and `array_ops.reshape`. The method takes a
    tensor and performs the correct transpose and reshape operation for a given
    set of indices. It returns the reshaped tensor as well as a list of indices
    necessary to reshape the tensor again after matrix multiplication.

    Args:
        a: `Tensor`.
        axes: List or `int32` `Tensor` of unique indices specifying valid axes of
         `a`.
        flipped: An optional `bool`. Defaults to `False`. If `True`, the method
            assumes that `a` is the second argument in the contraction operation.

    Returns:
        A tuple `(reshaped_a, free_dims, free_dims_static)` where `reshaped_a` is
        the tensor `a` reshaped to allow contraction via `matmul`, `free_dims` is
        either a list of integers or an `int32` `Tensor`, depending on whether
        the shape of a is fully specified, and free_dims_static is either a list
        of integers and None values, or None, representing the inferred
        static shape of the free dimensions
    """
    if a.get_shape().is_fully_defined() and isinstance(axes, (list, tuple)):
        shape_a = a.get_shape().as_list()
        axes = [i if i >= 0 else i + len(shape_a) for i in axes]
        free = [i for i in xrange(len(shape_a)) if i not in axes]
        free_dims = [shape_a[i] for i in free]
        prod_free = int(np.prod([shape_a[i] for i in free]))
        prod_axes = int(np.prod([shape_a[i] for i in axes]))
        perm = list(axes) + free if flipped else free + list(axes)
        new_shape = [prod_axes, prod_free] if flipped else [prod_free, prod_axes]
        reshaped_a = array_ops.reshape(array_ops.transpose(a, perm), new_shape)
        return reshaped_a, free_dims, free_dims
    else:
        if a.get_shape().ndims is not None and isinstance(axes, (list, tuple)):
            shape_a = a.get_shape().as_list()
            axes = [i if i >= 0 else i + len(shape_a) for i in axes]
            free = [i for i in xrange(len(shape_a)) if i not in axes]
            free_dims_static = [shape_a[i] for i in free]
        else:
            free_dims_static = None
        shape_a = array_ops.shape(a)
        rank_a = array_ops.rank(a)
        axes = ops.convert_to_tensor(axes, dtype=dtypes.int32, name="axes")
        axes = cast(axes >= 0, dtypes.int32) * axes + cast(
                axes < 0, dtypes.int32) * (
                        axes + rank_a)
        free, _ = array_ops.setdiff1d(range(rank_a), axes)
        free_dims = array_ops.gather(shape_a, free)
        axes_dims = array_ops.gather(shape_a, axes)
        prod_free_dims = reduce_prod(free_dims)
        prod_axes_dims = reduce_prod(axes_dims)
        perm = array_ops.concat([axes_dims, free_dims], 0)
        if flipped:
            perm = array_ops.concat([axes, free], 0)
            new_shape = array_ops.stack([prod_axes_dims, prod_free_dims])
        else:
            perm = array_ops.concat([free, axes], 0)
            new_shape = array_ops.stack([prod_free_dims, prod_axes_dims])
        reshaped_a = array_ops.reshape(array_ops.transpose(a, perm), new_shape)
        return reshaped_a, free_dims, free_dims_static


In [44]:
with ops.name_scope(name, "Tensordot", [a, b, axes]) as name:
    a = ops.convert_to_tensor(a, name="a")
    b = ops.convert_to_tensor(b, name="b")
    a_axes, b_axes = _tensordot_axes(a, axes)
    a_reshape, a_free_dims, a_free_dims_static = _tensordot_reshape(a, list(a_axes))
    b_reshape, b_free_dims, b_free_dims_static = _tensordot_reshape(
        b, list(b_axes), True)
    ab_matmul = tf.matmul(a_reshape, b_reshape)

In [45]:
ab_matmul

<tf.Tensor 'Tensordot_8/MatMul:0' shape=(3, 9) dtype=float32>

In [41]:
list(range(1,2))

[1]